In [ ]:
import requests

def get_api_version_info():
    url = "https://clinicaltrials.gov/api/v2/version"
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.json()
        else:
            return {"error": f"Failed to get version info, status code: {response.status_code}"}
    except requests.RequestException as e:
        return {"error": str(e)}

# Example usage
version_info = get_api_version_info()
print(version_info.get('apiVersion'))


: 

In [44]:
import requests

def fetch_clinical_trials(max_studies=5):
    url = "https://clinicaltrials.gov/api/v2/studies"
    page_token = None
    studies = []

    headers = {
        "Accept": "application/json"
    }

    while len(studies) < max_studies:
        params = {
            "query.titles": "cancer",
            "pageSize": int(2), # number of studies returned per page
            "pageToken": page_token,
            "format": "json",
            "markupFormat": "markdown"
        }

        response = requests.get(url, headers=headers, params=params)

        if response.status_code != 200:
            print(f"Error: Received status code {response.status_code}")
            print("Response content:", response.text)
            break

        try:
            data = response.json()
            print(f"Fetched {len(data.get('studies', []))} studies.")
        except requests.JSONDecodeError:
            print("Failed to decode JSON. Response was:", response.text)
            break

        # Process the response data if JSON decoding is successful
        for study in data.get("studies", []):
            if len(studies) >= max_studies:
                break

            study_info = {
                "nctId": study["protocolSection"]["identificationModule"]["nctId"],
                "briefTitle": study["protocolSection"]["identificationModule"]["briefTitle"],
                "organization": study["protocolSection"]["identificationModule"]["organization"]["fullName"],
                "statusVerifiedDate": study["protocolSection"]["statusModule"]["statusVerifiedDate"],
                "overallStatus": study["protocolSection"]["statusModule"]["overallStatus"],
            }
            studies.append(study_info)

        # Check for nextPageToken to continue pagination
        page_token = data.get("nextPageToken")
        if not page_token:
            break

    return studies

# Example call to test functionality
results = fetch_clinical_trials(max_studies=10)
for study in results:
    print(study)


Fetched 2 studies.
Fetched 2 studies.
Fetched 2 studies.
Fetched 2 studies.
Fetched 2 studies.
{'nctId': 'NCT01226589', 'briefTitle': 'Pharmacist Discharge Medication Reconciliation Study', 'organization': 'AHS Cancer Control Alberta', 'statusVerifiedDate': '2011-04', 'overallStatus': 'COMPLETED'}
{'nctId': 'NCT02839889', 'briefTitle': 'Tolerability, Safety, and Feasibility of Naloxegol in Patients With Cancer and OIC (Opioid Induced Constipation)', 'organization': 'Hospice of Henderson County, Inc.', 'statusVerifiedDate': '2019-02', 'overallStatus': 'TERMINATED'}
{'nctId': 'NCT01498289', 'briefTitle': 'S1201: Combination Chemo for Patients W/Advanced or Metastatic Esophageal, Gastric, or Gastroesophageal Junction Cancer', 'organization': 'SWOG Cancer Research Network', 'statusVerifiedDate': '2019-10', 'overallStatus': 'COMPLETED'}
{'nctId': 'NCT00256789', 'briefTitle': 'Once Weekly Radiation for Lung Cancer With Chemotherapy', 'organization': 'Clinical Oncology Research Associates', '

In [49]:
import firebase_admin
from firebase_admin import credentials, db

cred = credentials.Certificate('dukeai-103f8-369df2b50aa4.json')
firebase_admin.initialize_app(cred)

def store_to_firebase(studies):
    ref = db.reference("clinical_trials")  # Root reference in Firebase Realtime Database
    for study in studies:
        ref.push(study)  # Push each study as a new child in "clinical_trials" node


store_to_firebase(results)
print("Data stored in Firebase successfully.")

FileNotFoundError: [Errno 2] No such file or directory: 'dukeai-103f8-369df2b50aa4.json'